In [3]:
import mysql.connector
import re
from datetime import date

pattern_password = "^[a-zA-Z]+[1-9]+$"

class Ujian:
    cek_peserta_ujian = True
    nomor_soal = 1
    benar = 0
    cek_logins = False
    
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.today = date.today()

        #Fungsi disini untuk menghubungkan program ini dengan database
    def koneksi(self):
        self.mydb = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.database
        )
        self.mycursor = self.mydb.cursor()
        
        #Fungsi disini untuk mengecek apakah terdapat sebuah username di dalam database
    def cek_peserta(self):
        self.mycursor.execute("SELECT COUNT(*) FROM Data_Peserta")
        Result_Peserta = self.mycursor.fetchone()
        if Result_Peserta[0] > 0:
            self.cek_logins = True 
        else:
            self.cek_logins = False
        return Result_Peserta[0]
    
        #Fungsi ini digunakan untuk mengecek apakah username yang dimasukkan sudah melakukan ujian sebelumnya, 
        #atau tidak. Jika sudah maka username itu tidak bisa melakukan ujian lagi
    def cek_hasil_ujian(self, username):
        self.mycursor.execute(f"SELECT ID_Peserta FROM Data_Peserta WHERE username = '{username}'")
        ID = self.mycursor.fetchone()[0]
        self.mycursor.execute(f"SELECT COUNT(*) FROM Hasil_Ujian WHERE ID_Peserta = {ID}")
        count = self.mycursor.fetchone()[0]
        if count > 0:
            print("Anda sudah melakukan ujian sebelumnya!")
            return False
        else:
            return True
    
        #Fungsi disini digunakan untuk menutup database yang sudah dibuka sebelumnya
    def tutup_koneksi(self):
        self.mycursor.close()
        self.mydb.close()
        
        #Fungsi ini digunakan untuk mendaftarkan username dan password jika memang didalam database,
        #tidak ada username yang di input.
    def daftar(self):
        username = input("Masukkan username: ")
        Result_Peserta = self.cek_peserta()
        
        while True:
            password = input("Masukkan password: ")
            match = re.search(pattern_password, password)
            if match:
                query = "INSERT INTO Data_Peserta (ID_Peserta,Username, Password) VALUES (%s, %s, %s)"
                if Result_Peserta == 0:
                    ID = "1"
                    ID = int(ID)
                elif Result_Peserta > 0:
                    ID = f'{Result_Peserta}'
                    ID = int(ID) + 1
                value = (ID,username, password)
                self.mycursor.execute(query, value)
                self.mydb.commit()
                print("Pendaftaran berhasil.")
                break
                
            else:
                if password.isalpha() == False and password.isdigit() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 angka atau 1 huruf")
                elif password.isdigit() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 angka")
                elif password.isalpha() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 huruf")

    # Setelah kita mendaftar pada fungsi daftar disini kita login untuk bisa mengerjakan ujian
    def login(self,username):
        while True:
            self.username = username
            password = input("Masukkan password: ")
            query = "SELECT * FROM Data_Peserta WHERE username = %s AND password = %s"
            value = (self.username, password)
            self.mycursor.execute(query, value)
            result = self.mycursor.fetchone()
            if result:
                print("Login berhasil.\n")
                break
            else:
                print("Username atau password salah, silakan coba lagi.")
    
    #Fungsi disini untuk mengetahui jumlah soal yang ada di database ada berapa
    def jumlah_soal(self):
        self.mycursor.execute("SELECT COUNT(*) FROM Soal_Ujian")
        result = self.mycursor.fetchone()
        self.jumlah_soal = result[0]
    
    #Fungsi ini digunakan untuk merandom soal yang nanti kita tampilkan pada output
    def soal_random(self):
        self.mycursor.execute("SELECT * FROM Soal_Ujian ORDER BY RAND() LIMIT %s", (self.jumlah_soal,))
        self.selected_rows = self.mycursor.fetchall()
    

    def tampilkan_soal(self,file_output):
        with open(file_output, "w") as f:
            for row in self.selected_rows:
                print("{}. {}".format(self.nomor_soal, row[0]))
                print("   A. {}".format(row[1]))
                print("   B. {}".format(row[2]))
                print("   C. {}".format(row[3]))
                print("   D. {}".format(row[4]))
                print("   E. {}".format(row[5]))
                
                f.write("{}. {}\n".format(self.nomor_soal, row[0]))
                f.write("   A. {}\n".format(row[1]))
                f.write("   B. {}\n".format(row[2]))
                f.write("   C. {}\n".format(row[3]))
                f.write("   D. {}\n".format(row[4]))
                f.write("   E. {}\n".format(row[5]))

                while True:
                    try:
                        jawaban = input("Jawaban Anda (A/B/C/D/E): ").upper()
                        if jawaban not in ["A", "B", "C", "D", "E"]:
                            raise ValueError
                        break
                    except ValueError:
                        print("Pilihan jawaban salah, silakan coba lagi.")
                        continue
                        
                if jawaban == row[6]:
                    print("Jawaban Anda benar!\n")
                    f.write("Jawaban Anda: {}\n".format(jawaban))
                    f.write("Jawaban benar!\n\n")
                    self.benar += 1
                else:
                    print("Jawaban Anda salah.\n")
                    f.write("Jawaban Anda: {}\n".format(jawaban))
                    f.write("Jawaban salah.\n\n")
                self.nomor_soal += 1
            
    #Fungsi disini digunakan menampilkan sekaligus menyimpan hasil ujian dari user
    def tampilkan_hasil(self, username, file_output):
        print("Hasil ujian:")
        print("Jumlah soal benar: {}".format(self.benar))
        print("Jumlah soal salah: {}".format(self.jumlah_soal - self.benar))
        print("Skor Anda: {}%".format(self.benar/self.jumlah_soal*100))

        # Menuliskan hasil ke dalam file
        with open(file_output, 'a') as f:
            f.write("Hasil ujian: \n")
            f.write("Nama Peserta: {}\n".format(username))
            f.write("Jumlah soal benar: {}\n".format(self.benar))
            f.write("Jumlah soal salah: {}\n".format(self.jumlah_soal - self.benar))
            f.write("Skor Anda: {}%\n\n".format(self.benar/self.jumlah_soal*100))

        self.mycursor.execute(f"SELECT ID_Peserta FROM Data_Peserta WHERE username = '{username}'")
        ID = self.mycursor.fetchone()[0]
        query = "INSERT INTO Hasil_Ujian (ID_Peserta,Soal_Benar, Total_Soal, Nilai, Tanggal_Ujian) VALUES (%s, %s, %s, %s, %s)"
        value = (ID,self.benar, self.jumlah_soal, self.benar/self.jumlah_soal*100, self.today)
        self.mycursor.execute(query, value)
        self.mydb.commit()
    
if __name__ == '__main__':
    ujian = Ujian("localhost", "root", "", "pemrog2_pas")
    ujian.koneksi()
    ujian.cek_peserta()
    
    while True:
        print("Pilih menu:\n1. Login \n2. Daftar")
        menu = input()
        if menu == "1":
            if ujian.cek_logins == False:
                print("Username belum terdaftar!")
                print("Silakan pilih menu 2 untuk mendaftar.\n")
                pass
            else:
                username = input("Masukkan username: ")
                ujian.login(username)
                if ujian.cek_hasil_ujian(username) == False:
                    break
                ujian.jumlah_soal()
                ujian.soal_random()
                ujian.tampilkan_soal('Hasil_Ujian_'+f'{username}'+'.txt')
                ujian.tampilkan_hasil(username,'Hasil_Ujian_'+f'{username}'+'.txt')
                ujian.tutup_koneksi()  
                break
                
        elif menu == "2":
            ujian.daftar()
            break
        else:
            print("Masukkan opsi yang benar\n")

Pilih menu:
1. Login 
2. Daftar
1
Masukkan username: powe
Masukkan password: weq21
Login berhasil.

Anda sudah melakukan ujian sebelumnya!


In [ ]:
import mysql.connector
from datetime import date

class Database:
    def __init__(self, host, user, password, database):
        self.__host = host
        self.__user = user
        self.__password = password
        self.__database = database

    def connect(self):
        self.mydb = mysql.connector.connect(
            host=self._Database__host,
            user=self._Database__user,
            password=self._Database__password,
            database=self._Database__database
        )
        self.mycursor = self.mydb.cursor()

    def close(self):
        self.mycursor.close()
        self.mydb.close()

class Ujian(Database):
    def __init__(self, host, user, password, database):
        super().__init__(host, user, password, database)
        self.today = date.today()
        self.cek_logins = False
        self.jumlah_soal = 0
        self.nomor_soal = 1
        self.benar = 0

    def cek_peserta(self):
        self.mycursor.execute("SELECT COUNT(*) FROM Data_Peserta")
        Result_Peserta = self.mycursor.fetchone()
        if Result_Peserta[0] > 0:
            self.cek_logins = True
        else:
            self.cek_logins = False
        return Result_Peserta[0]

    def cek_hasil_ujian(self, username):
        self.mycursor.execute(f"SELECT ID_Peserta FROM Data_Peserta WHERE username = '{username}'")
        ID = self.mycursor.fetchone()[0]
        self.mycursor.execute(f"SELECT COUNT(*) FROM Hasil_Ujian WHERE ID_Peserta = {ID}")
        count = self.mycursor.fetchone()[0]
        if count > 0:
            print("Anda sudah melakukan ujian sebelumnya!")
            return False
        else:
            return True

    def daftar(self):
        username = input("Masukkan username: ")
        Result_Peserta = self.cek_peserta()

        while True:
            password = input("Masukkan password: ")
            match = re.search(pattern_password, password)
            if match:
                query = "INSERT INTO Data_Peserta (ID_Peserta,Username, Password) VALUES (%s, %s, %s)"
                if Result_Peserta == 0:
                    ID = "1"
                    ID = int(ID)
                elif Result_Peserta > 0:
                    ID = f'{Result_Peserta}'
                    ID = int(ID) + 1
                value = (ID, username, password)
                self.mycursor.execute(query, value)
                self.mydb.commit()
                print("Pendaftaran berhasil.")
                break

            else:
                if password.isalpha() == False and password.isdigit() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 angka atau 1 huruf")
                elif password.isdigit() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 angka")
                elif password.isalpha() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 huruf")
    def login(self, username):
        while True:
            self.username = username
            password = input("Masukkan password: ")
            query = "SELECT * FROM Data_Peserta WHERE username = %s AND password = %s"
            value = (self.username, password)
            self.mycursor.execute(query, value)
            result = self.mycursor.fetchone()
            if result:
                print("Login berhasil.\n")
                break
            else:
                print("Username atau password salah, silakan coba lagi.")

    def jumlah_soal(self):
        self.mycursor.execute("SELECT COUNT(*) FROM Soal_Ujian")
        result = self.mycursor.fetchone()
        self.jumlah_soal = result[0]

    def soal_random(self):
        self.mycursor.execute("SELECT * FROM Soal_Ujian ORDER BY RAND() LIMIT %s", (self.jumlah_soal,))
        self.selected_rows = self.mycursor.fetchall()

    def tampilkan_soal(self, file_output):
        with open(file_output, "w") as f:
            for row in self.selected_rows:
                print("{}. {}".format(self.nomor_soal, row[0]))
                print("   A. {}".format(row[1]))
                print("   B. {}".format(row[2]))
                print("   C. {}".format(row[3]))
                print("   D. {}".format(row[4]))
                print("   E. {}".format(row[5]))

                f.write("{}. {}\n".format(self.nomor_soal, row[0]))
                f.write("   A. {}\n".format(row[1]))
                f.write("   B. {}\n".format(row[2]))
                f.write("   C. {}\n".format(row[3]))
                f.write("   D. {}\n".format(row[4]))
                f.write("   E. {}\n".format(row[5]))

                while True:
                    try:
                        jawaban = input("Jawaban Anda (A/B/C/D/E): ").upper()
                        if jawaban not in ["A", "B", "C", "D", "E"]:
                            raise ValueError
                        break
                    except ValueError:
                        print("Pilihan jawaban salah, silakan coba lagi.")
                        continue

                if jawaban == row[6]:
                    print("Jawaban Anda benar!\n")
                    f.write("Jawaban Anda: {}\n".format(jawaban))
                    f.write("Jawaban benar!\n\n")
                    self.benar += 1
                else:
                    print("Jawaban Anda salah.\n")
                    f.write("Jawaban Anda: {}\n".format(jawaban))
                    f.write("Jawaban salah.\n\n")
                self.nomor_soal += 1

    def tampilkan_hasil(self, username, file_output):
        print("Hasil ujian:")
        print("Jumlah soal benar: {}".format(self.benar))
        print("Jumlah soal salah: {}".format(self.jumlah_soal - self.benar))
        print("Skor Anda: {}%".format(self.benar / self.jumlah_soal * 100))

        with open(file_output, 'a') as f:
            f.write("Hasil ujian: \n")
            f.write("Nama Peserta: {}\n".format(username))
            f.write("Jumlah soal benar: {}\n".format(self.benar))
            f.write("Jumlah soal salah: {}\n".format(self.jumlah_soal - self.benar))
            f.write("Skor Anda: {}%\n\n".format(self.benar / self.jumlah_soal * 100))

        self.mycursor.execute(f"SELECT ID_Peserta FROM Data_Peserta WHERE username = '{username}'")
        ID = self.mycursor.fetchone()[0]
        query = "INSERT INTO Hasil_Ujian (ID_Peserta, Soal_Benar, Total_Soal, Nilai, Tanggal_Ujian) VALUES (%s, %s, %s, %s, %s)"
        value = (ID, self.benar, self.jumlah_soal, self.benar / self.jumlah_soal * 100, self.today)
        self.mycursor.execute(query, value)
        self.mydb.commit()
if __name__ == '__main__':
    ujian = Ujian("localhost", "root", "", "pemrog2_pas")
    ujian.koneksi()
    ujian.cek_peserta()
    
    while True:
        print("Pilih menu:\n1. Login \n2. Daftar")
        menu = input()
        if menu == "1":
            if ujian.cek_logins == False:
                print("Username belum terdaftar!")
                print("Silakan pilih menu 2 untuk mendaftar.\n")
                pass
            else:
                username = input("Masukkan username: ")
                ujian.login(username)
                if ujian.cek_hasil_ujian(username) == False:
                    break
                ujian.jumlah_soal()
                ujian.soal_random()
                ujian.tampilkan_soal('Hasil_Ujian_'+f'{username}'+'.txt')
                ujian.tampilkan_hasil(username,'Hasil_Ujian_'+f'{username}'+'.txt')
                ujian.tutup_koneksi()  
                break
                
        elif menu == "2":
            ujian.daftar()
            break
        else:
            print("Masukkan opsi yang benar\n")

In [1]:
import mysql.connector
from datetime import date

class Database:
    def __init__(self, host, user, password, database):
        self.__host = host
        self.__user = user
        self.__password = password
        self.__database = database

    def connect(self):
        self.mydb = mysql.connector.connect(
            host=self._Database__host,
            user=self._Database__user,
            password=self._Database__password,
            database=self._Database__database
        )
        self.mycursor = self.mydb.cursor()

    def close(self):
        self.mycursor.close()
        self.mydb.close()

class Ujian:
    def __init__(self, database):
        self.db = database
        self.today = date.today()
        self.cek_logins = False
        self.jumlah_soal = 0
        self.nomor_soal = 1
        self.benar = 0

    def cek_peserta(self):
        self.mycursor.execute("SELECT COUNT(*) FROM Data_Peserta")
        Result_Peserta = self.mycursor.fetchone()
        if Result_Peserta[0] > 0:
            self.cek_logins = True
        else:
            self.cek_logins = False
        return Result_Peserta[0]

    def cek_hasil_ujian(self, username):
        self.mycursor.execute(f"SELECT ID_Peserta FROM Data_Peserta WHERE username = '{username}'")
        ID = self.mycursor.fetchone()[0]
        self.mycursor.execute(f"SELECT COUNT(*) FROM Hasil_Ujian WHERE ID_Peserta = {ID}")
        count = self.mycursor.fetchone()[0]
        if count > 0:
            print("Anda sudah melakukan ujian sebelumnya!")
            return False
        else:
            return True

    def daftar(self):
        username = input("Masukkan username: ")
        Result_Peserta = self.cek_peserta()

        while True:
            password = input("Masukkan password: ")
            match = re.search(pattern_password, password)
            if match:
                query = "INSERT INTO Data_Peserta (ID_Peserta,Username, Password) VALUES (%s, %s, %s)"
                if Result_Peserta == 0:
                    ID = "1"
                    ID = int(ID)
                elif Result_Peserta > 0:
                    ID = f'{Result_Peserta}'
                    ID = int(ID) + 1
                value = (ID, username, password)
                self.mycursor.execute(query, value)
                self.mydb.commit()
                print("Pendaftaran berhasil.")
                break

            else:
                if password.isalpha() == False and password.isdigit() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 angka atau 1 huruf")
                elif password.isdigit() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 angka")
                elif password.isalpha() == False:
                    print(f"Password yang dimasukkan tidak memenuhi ketentuan, karena tidak terdapat 1 huruf")
    def login(self, username):
        while True:
            self.username = username
            password = input("Masukkan password: ")
            query = "SELECT * FROM Data_Peserta WHERE username = %s AND password = %s"
            value = (self.username, password)
            self.mycursor.execute(query, value)
            result = self.mycursor.fetchone()
            if result:
                print("Login berhasil.\n")
                break
            else:
                print("Username atau password salah, silakan coba lagi.")

    def jumlah_soal(self):
        self.mycursor.execute("SELECT COUNT(*) FROM Soal_Ujian")
        result = self.mycursor.fetchone()
        self.jumlah_soal = result[0]

    def soal_random(self):
        self.mycursor.execute("SELECT * FROM Soal_Ujian ORDER BY RAND() LIMIT %s", (self.jumlah_soal,))
        self.selected_rows = self.mycursor.fetchall()

    def tampilkan_soal(self, file_output):
        with open(file_output, "w") as f:
            for row in self.selected_rows:
                print("{}. {}".format(self.nomor_soal, row[0]))
                print("   A. {}".format(row[1]))
                print("   B. {}".format(row[2]))
                print("   C. {}".format(row[3]))
                print("   D. {}".format(row[4]))
                print("   E. {}".format(row[5]))

                f.write("{}. {}\n".format(self.nomor_soal, row[0]))
                f.write("   A. {}\n".format(row[1]))
                f.write("   B. {}\n".format(row[2]))
                f.write("   C. {}\n".format(row[3]))
                f.write("   D. {}\n".format(row[4]))
                f.write("   E. {}\n".format(row[5]))

                while True:
                    try:
                        jawaban = input("Jawaban Anda (A/B/C/D/E): ").upper()
                        if jawaban not in ["A", "B", "C", "D", "E"]:
                            raise ValueError
                        break
                    except ValueError:
                        print("Pilihan jawaban salah, silakan coba lagi.")
                        continue

                if jawaban == row[6]:
                    print("Jawaban Anda benar!\n")
                    f.write("Jawaban Anda: {}\n".format(jawaban))
                    f.write("Jawaban benar!\n\n")
                    self.benar += 1
                else:
                    print("Jawaban Anda salah.\n")
                    f.write("Jawaban Anda: {}\n".format(jawaban))
                    f.write("Jawaban salah.\n\n")
                self.nomor_soal += 1

    def tampilkan_hasil(self, username, file_output):
        print("Hasil ujian:")
        print("Jumlah soal benar: {}".format(self.benar))
        print("Jumlah soal salah: {}".format(self.jumlah_soal - self.benar))
        print("Skor Anda: {}%".format(self.benar / self.jumlah_soal * 100))

        with open(file_output, 'a') as f:
            f.write("Hasil ujian: \n")
            f.write("Nama Peserta: {}\n".format(username))
            f.write("Jumlah soal benar: {}\n".format(self.benar))
            f.write("Jumlah soal salah: {}\n".format(self.jumlah_soal - self.benar))
            f.write("Skor Anda: {}%\n\n".format(self.benar / self.jumlah_soal * 100))

        self.mycursor.execute(f"SELECT ID_Peserta FROM Data_Peserta WHERE username = '{username}'")
        ID = self.mycursor.fetchone()[0]
        query = "INSERT INTO Hasil_Ujian (ID_Peserta, Soal_Benar, Total_Soal, Nilai, Tanggal_Ujian) VALUES (%s, %s, %s, %s, %s)"
        value = (ID, self.benar, self.jumlah_soal, self.benar / self.jumlah_soal * 100, self.today)
        self.mycursor.execute(query, value)
        self.mydb.commit()
if __name__ == '__main__':
    ujian = Ujian("localhost", "root", "", "pemrog2_pas")
    ujian.koneksi()
    ujian.cek_peserta()
    
    while True:
        print("Pilih menu:\n1. Login \n2. Daftar")
        menu = input()
        if menu == "1":
            if ujian.cek_logins == False:
                print("Username belum terdaftar!")
                print("Silakan pilih menu 2 untuk mendaftar.\n")
                pass
            else:
                username = input("Masukkan username: ")
                ujian.login(username)
                if ujian.cek_hasil_ujian(username) == False:
                    break
                ujian.jumlah_soal()
                ujian.soal_random()
                ujian.tampilkan_soal('Hasil_Ujian_'+f'{username}'+'.txt')
                ujian.tampilkan_hasil(username,'Hasil_Ujian_'+f'{username}'+'.txt')
                ujian.tutup_koneksi()  
                break
                
        elif menu == "2":
            ujian.daftar()
            break
        else:
            print("Masukkan opsi yang benar\n")

Judul: Pemrograman Python
Penulis: John Doe
John Doe sedang menulis.
